In [4]:
# -*- coding: utf-8 -*-
"""
Created on Fri Sep 28 12:03:21 2018

@author: seraj
"""

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorboard.plugins import projector
from tensorflow.examples.tutorials.mnist import input_data

PATH = os.getcwd()

LOG_DIR = '/Users/denisdunn/Desktop/'+'full_aerin'
#LOG_DIR = PATH + '/Users/denisdunn/Desktop/log-1'

embed_count = 1600
mnist = input_data.read_data_sets(PATH + "/data/")
# mnist.train, mnist.test, mnist.validation
batch_xs, batch_ys = mnist.train.next_batch(1600)
x_test = batch_xs/255
y_test = batch_ys
batch_ys.shape[0]
#x_test.shape[0] # the number of images in embed (sprite.PNG)that is number of rows in csv
#x_test.shape[1] # sqrt is the dim of one image that in one row = 28 * 28 
train_generator = train_datagen.flow_from_directory(
        directory='/Users/denisdunn/Desktop/'+'full_aerin',
        target_size=(IMG_SIZE, IMG_SIZE),
        color_mode='rgb',
        batch_size=1,
        class_mode='categorical')
#logdir = r'logdir'  # you will need to change this!!!
logdir = '/Users/denisdunn/Desktop/logs'  # you will need to change this!!!
# setup the write and embedding tensor

summary_writer = tf.summary.FileWriter(logdir)
#summary_writer: The summary writer used for writing events. config:
embedding_var = tf.Variable(x_test, name='fmnist_embedding') # embedding variable

config = projector.ProjectorConfig() # use the projector
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name # embedding tensor

embedding.metadata_path = os.path.join(logdir, 'metadata.tsv')
embedding.sprite.image_path = os.path.join(logdir, 'sprite.png')
embedding.sprite.single_image_dim.extend([28, 28]) # size of single image in the sprite

projector.visualize_embeddings(summary_writer, config) # configuire projector

# run the sesion to create the model check point

with tf.Session() as sesh:  # tensorflow session
    sesh.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.save(sesh, os.path.join(logdir, 'model.ckpt'))
    
    
# create the sprite image and the metadata file

rows = 28
cols = 28

label = ['0','1', '2', '3', '4', '5', '6', '7', '8', '9'] # mape the numbers lable to its name

sprite_dim = int(np.sqrt(x_test.shape[0])) #dim of  sprite.png is sqrt (number of row in csv)

sprite_image = np.ones((cols * sprite_dim, rows * sprite_dim)) # creat blank(temblate) sprite_image

index = 0
labels = []
for i in range(sprite_dim):
    for j in range(sprite_dim):
        
        labels.append(label[int(y_test[index])]) # get what is image(get its number that is lable in csv) the map this number to lable name 
        
        sprite_image[
            i * cols: (i + 1) * cols,
            j * rows: (j + 1) * rows
        ] = x_test[index].reshape(28, 28) * -1 + 1
        
        index += 1
        
        
with open(embedding.metadata_path, 'w') as meta:
    meta.write('Index\tLabel\n')
    for index, label in enumerate(labels):
        meta.write('{}\t{}\n'.format(index, label))
        
plt.imsave(embedding.sprite.image_path, sprite_image, cmap='gray')
plt.imshow(sprite_image, cmap='gray')
plt.show()


ModuleNotFoundError: No module named 'tensorflow.examples.tutorials'

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorboard.plugins import projector

def register_embedding(embedding_tensor_name, meta_data_fname, log_dir):
    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = embedding_tensor_name
    embedding.metadata_path = meta_data_fname
    projector.visualize_embeddings(log_dir, config)

def get_random_data(shape=(100,100)):
    x = np.random.rand(*shape)
    y = np.random.randint(low=0, high=2, size=shape[0])
    return x, y

def save_labels_tsv(labels, filepath, log_dir):
    with open(os.path.join(log_dir, filepath), 'w') as f:
        for label in labels:
            f.write('{}\n'.format(label))

LOG_DIR = 'tmp'  # Tensorboard log dir
META_DATA_FNAME = 'meta.tsv'  # Labels will be stored here
EMBEDDINGS_TENSOR_NAME = 'embeddings'
EMBEDDINGS_FPATH = os.path.join(LOG_DIR, EMBEDDINGS_TENSOR_NAME + '.ckpt')
STEP = 0

x, y = get_random_data((100,100))
register_embedding(EMBEDDINGS_TENSOR_NAME, META_DATA_FNAME, LOG_DIR)
save_labels_tsv(y, META_DATA_FNAME, LOG_DIR)
# Size of files created on disk: 80.5kB
tensor_embeddings = tf.Variable(x, name=EMBEDDINGS_TENSOR_NAME)
saver = tf.compat.v1.train.Saver([tensor_embeddings])  # Must pass list or dict
saver.save(sess=None, global_step=STEP, save_path=EMBEDDINGS_FPATH)